In [1]:
# @title TEST B — Prime/φ Bridge & Zeta-Flavored Audit (press ▶ to run)
# @markdown This cell:
# @markdown - Builds primes up to PRIME_MAX (default 10,000,000)
# @markdown - Checks whether 4π Σ_{p≤P} p^{-φ} is nearest to **9** as P grows
# @markdown - Fits the best exponent ŝ_P for k=9 at each P (coarse grid + golden-section refine)
# @markdown - Adds a simple second-order correction (+½ Σ p^{-2s}) and a crude tail bound
# @markdown - Runs a neighborhood test (s near φ) to see how selective k=9 is
# @markdown Outputs: CSVs + concise printed summaries.

import os, math, time, numpy as np, pandas as pd
from typing import List, Tuple, Dict

# ===============================
# CONFIG — tweak to taste
# ===============================
CONFIG = dict(
    OUTDIR="/content",
    PRIME_MAX=10_000_000,            # up to 10M primes (≈664k primes) — OK in Colab; raise if you like
    P_VALUES=[100_000, 200_000, 500_000, 1_000_000, 2_000_000, 5_000_000, 10_000_000],
    K_RANGE=list(range(6, 13)),      # k candidates: 6..12 (includes 9)
    S_AT_PHI_WINDOW=0.08,            # for best-s search brackets around φ
    S_GRID_COARSE_STEP=0.001,        # coarse grid step for ŝ search (refined by golden section)
    GOLDEN_ITERS=40,                 # golden-section refinements per P
    NEIGHBOR_SAMPLES=200,            # neighborhood Monte Carlo samples around φ
    NEIGHBOR_DELTA=0.02,             # sample s ~ Uniform[φ-Δ, φ+Δ]
    RANDOM_SEED=42
)

# ===============================
# UTILITIES
# ===============================
def now(): return time.strftime('%Y-%m-%d %H:%M:%S')

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def sieve_primes(n: int) -> np.ndarray:
    """Classic fast sieve, returns ascending primes up to n (inclusive)."""
    bs = np.ones(n + 1, dtype=np.bool_)
    bs[:2] = False
    limit = int(n ** 0.5) + 1
    for p in range(2, limit):
        if bs[p]:
            bs[p*p:n+1:p] = False
    return np.flatnonzero(bs)

def sum_p_pow_s(logp: np.ndarray, s: float, idx_end: int) -> float:
    """Compute Σ_{i<idx_end} exp(-s log p_i) in float64."""
    # Equivalent to sum(p[:idx_end] ** (-s)) but more stable/fast
    return float(np.exp(-s * logp[:idx_end]).sum(dtype=np.float64))

def nearest_integer(x: float) -> Tuple[int, float]:
    k = int(round(x))
    return k, abs(x - k)

def golden_section_minimize(f, a, b, iters=40):
    """Golden-section search on [a,b] to minimize f(s). Returns (s*, f(s*))."""
    gr = (math.sqrt(5) + 1) / 2
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    fc = f(c); fd = f(d)
    for _ in range(iters):
        if fc < fd:
            b, d, fd = d, c, fc
            c = b - (b - a) / gr
            fc = f(c)
        else:
            a, c, fc = c, d, fd
            d = a + (b - a) / gr
            fd = f(d)
    s_best = (a + b) / 2
    return s_best, f(s_best)

def crude_tail_bound(P: float, s: float) -> float:
    """
    Very crude bound for Σ_{p>P} p^{-s} using
    ∫_P^∞ x^{-s} / log x dx  ≲  (1/log P) * ∫_P^∞ x^{-s} dx = P^{1-s} / ((s-1) log P)
    (valid for s>1, P>e). This is an overestimate — we use it as a conservative error bar.
    """
    if s <= 1 or P <= math.e: return float('inf')
    return P**(1.0 - s) / ((s - 1.0) * math.log(P))

# ===============================
# MAIN
# ===============================
def main():
    cfg = CONFIG
    ensure_dir(cfg["OUTDIR"])
    np.random.seed(cfg["RANDOM_SEED"])

    print(f"[{now()}] Sieve primes to {cfg['PRIME_MAX']:,} …")
    primes = sieve_primes(cfg["PRIME_MAX"]).astype(np.int64)
    logp = np.log(primes.astype(np.float64))
    print(f"  primes found: {len(primes):,} (first: {primes[0]}, last: {primes[-1]})")

    # constants
    phi = (1.0 + math.sqrt(5.0)) / 2.0
    four_pi = 4.0 * math.pi
    target_k9 = 9.0
    target_9_over_4pi = target_k9 / four_pi

    # map P_values to indices in the prime list
    P_vals = []
    idx_vals = []
    for Pv in cfg["P_VALUES"]:
        i = int(np.searchsorted(primes, Pv, side='right'))
        if i == 0:
            continue
        P_vals.append(Pv)
        idx_vals.append(i)
    print(f"  Using {len(P_vals)} P cutoffs: {P_vals}")

    # -------------------------------
    # B2. Integer uniqueness at s=phi
    # -------------------------------
    rows_phi = []
    for Pv, idx_end in zip(P_vals, idx_vals):
        S_phi = sum_p_pow_s(logp, phi, idx_end)
        x = four_pi * S_phi
        for k in CONFIG["K_RANGE"]:
            rows_phi.append(dict(P=Pv, s="phi", k=int(k), value=x, distance=abs(x - k)))
    df_phi = pd.DataFrame(rows_phi)
    df_phi.to_csv(os.path.join(cfg["OUTDIR"], "integer_k_scan_at_phi.csv"), index=False)
    print("\nNearest integer at s=phi (by P):")
    for Pv in P_vals:
        sub = df_phi[df_phi.P == Pv].sort_values("distance").head(3)
        k0 = int(sub.iloc[0]["k"])
        dist0 = sub.iloc[0]["distance"]
        print(f"  P≤{Pv:>9,}: nearest k={k0} (gap={dist0:.6e}); next: k={int(sub.iloc[1]['k'])}, k={int(sub.iloc[2]['k'])}")

    # -----------------------------------------
    # B3. Best exponent ŝ_P for k=9 (per P)
    # -----------------------------------------
    def best_s_for_k9(idx_end: int) -> Tuple[float, float, float]:
        """Return (s_best, min_abs_err, f_calls) for S(s)≈9/(4π) over primes[:idx_end]."""
        # coarse bracket around phi
        a = phi - CONFIG["S_AT_PHI_WINDOW"]
        b = phi + CONFIG["S_AT_PHI_WINDOW"]
        # coarse scan
        s_vals = np.arange(a, b + 1e-12, CONFIG["S_GRID_COARSE_STEP"])
        errs = []
        for s in s_vals:
            S = sum_p_pow_s(logp, float(s), idx_end)
            errs.append(abs(S - target_9_over_4pi))
        j = int(np.argmin(errs))
        s0 = float(s_vals[max(0, j-1)])
        s1 = float(s_vals[min(len(s_vals)-1, j+1)])
        # refine with golden section on [s0,s1], padding if needed
        pad = 0.01
        left = max(a, s0 - pad)
        right = min(b, s1 + pad)
        def f(s):
            return abs(sum_p_pow_s(logp, float(s), idx_end) - target_9_over_4pi)
        s_best, err_best = golden_section_minimize(f, left, right, iters=CONFIG["GOLDEN_ITERS"])
        return s_best, err_best, float(len(s_vals) + CONFIG["GOLDEN_ITERS"])

    rows_best = []
    for Pv, idx_end in zip(P_vals, idx_vals):
        s_best, err_best, calls = best_s_for_k9(idx_end)
        rows_best.append(dict(P=Pv, s_best=s_best, s_best_minus_phi=s_best - phi, abs_err=err_best, evals=int(calls)))
    df_best = pd.DataFrame(rows_best)
    df_best.to_csv(os.path.join(cfg["OUTDIR"], "s_best_for_k9.csv"), index=False)
    print("\nBest exponent ŝ_P for k=9 (coarse+golden):")
    print(df_best.to_string(index=False, justify='right', float_format=lambda z: f"{z:.9f}"))

    # -------------------------------------------------------
    # B4. Second-order correction & crude tail upper bounds
    # -------------------------------------------------------
    rows_corr = []
    for Pv, idx_end in zip(P_vals, idx_vals):
        # raw sums
        S_phi = sum_p_pow_s(logp, phi, idx_end)
        S2_phi = 0.5 * sum_p_pow_s(logp, 2.0*phi, idx_end)  # half of p^{-2s}
        S_corr = S_phi + S2_phi

        # crude tail bounds
        tail1 = crude_tail_bound(float(Pv), phi)
        tail2 = 0.5 * crude_tail_bound(float(Pv), 2.0*phi)
        tail_tot = tail1 + tail2

        x_raw = four_pi * S_phi
        x_corr = four_pi * S_corr

        k_raw, gap_raw = nearest_integer(x_raw)
        k_corr, gap_corr = nearest_integer(x_corr)

        rows_corr.append(dict(
            P=Pv,
            fourpi_S_raw=x_raw, nearest_k_raw=k_raw, gap_raw=gap_raw,
            fourpi_S_corr=x_corr, nearest_k_corr=k_corr, gap_corr=gap_corr,
            tail_bound_total=tail_tot
        ))
    df_corr = pd.DataFrame(rows_corr)
    df_corr.to_csv(os.path.join(cfg["OUTDIR"], "phi_k9_corrections_and_tails.csv"), index=False)
    print("\nφ with second-order correction (+½Σp^{-2s}) and crude tail bounds (upper):")
    for _, r in df_corr.iterrows():
        print(f"  P≤{int(r.P):>9,}: raw→k={int(r.nearest_k_raw)} gap={r.gap_raw:.3e}; "
              f"corr→k={int(r.nearest_k_corr)} gap={r.gap_corr:.3e}; tail≲{r.tail_bound_total:.3e}")

    # ---------------------------------------
    # B5. Neighborhood test around φ
    # ---------------------------------------
    np.random.seed(CONFIG["RANDOM_SEED"])
    s_samples = np.random.uniform(phi - CONFIG["NEIGHBOR_DELTA"], phi + CONFIG["NEIGHBOR_DELTA"], size=CONFIG["NEIGHBOR_SAMPLES"])
    idx_full = idx_vals[-1]  # use largest P
    ks = CONFIG["K_RANGE"]
    rows_nb = []
    for s in s_samples:
        S = sum_p_pow_s(logp, float(s), idx_full)
        x = four_pi * S
        k_near, gap = nearest_integer(x)
        rows_nb.append(dict(s=float(s), nearest_k=k_near, gap=gap))
    df_nb = pd.DataFrame(rows_nb)
    df_nb.to_csv(os.path.join(cfg["OUTDIR"], "neighborhood_phi_hist.csv"), index=False)
    counts = df_nb["nearest_k"].value_counts().sort_index()
    print("\nNeighborhood test near φ at max P (counts of nearest k to 4πΣp^{-s}):")
    for k in ks:
        print(f"  k={k}: {int(counts.get(k,0))} / {len(df_nb)}")

    print(f"\nDone. Files in {CONFIG['OUTDIR']}:")
    for fn in [
        "integer_k_scan_at_phi.csv",
        "s_best_for_k9.csv",
        "phi_k9_corrections_and_tails.csv",
        "neighborhood_phi_hist.csv",
    ]:
        print(" ", fn)

if __name__ == "__main__":
    print(f"[{now()}] Starting TEST B …")
    main()


[2025-10-03 19:14:42] Starting TEST B …
[2025-10-03 19:14:42] Sieve primes to 10,000,000 …
  primes found: 664,579 (first: 2, last: 9999991)
  Using 7 P cutoffs: [100000, 200000, 500000, 1000000, 2000000, 5000000, 10000000]

Nearest integer at s=phi (by P):
  P≤  100,000: nearest k=9 (gap=7.132235e-03); next: k=10, k=8
  P≤  200,000: nearest k=9 (gap=7.617641e-03); next: k=10, k=8
  P≤  500,000: nearest k=9 (gap=7.986173e-03); next: k=10, k=8
  P≤1,000,000: nearest k=9 (gap=8.144333e-03); next: k=10, k=8
  P≤2,000,000: nearest k=9 (gap=8.242502e-03); next: k=10, k=8
  P≤5,000,000: nearest k=9 (gap=8.317657e-03); next: k=10, k=8
  P≤10,000,000: nearest k=9 (gap=8.350219e-03); next: k=10, k=8

Best exponent ŝ_P for k=9 (coarse+golden):
       P      s_best  s_best_minus_phi     abs_err  evals
  100000 1.618615961       0.000581972 0.000000000    201
  200000 1.618655305       0.000621316 0.000000000    201
  500000 1.618685139       0.000651150 0.000000000    201
 1000000 1.618697929    